# Data Cleaning

In [80]:
#Importing relevant libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [81]:
# Importing our data file
df_original = pd.read_csv('../Data/AviationData.csv', encoding='latin1', low_memory=False)

# Making a copy of the original dataframe in case I need it
df = df_original.copy()

# Listing the columns out to for pruning columns that are irrelevant to our goal
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

## Notes on objectives and more

- Find the top 5 or 10 make and model of airplanes in terms of safety
- Safety metrics by make/model: accident rate, fatality rate, injury rate
- Consider exploring safety metrics by engine type
- Think about weather/phase of flight only if there is time
- Number of passengers = fatally injured + injured + uninjured
- Determine size of airplane based on passenger number (look up how planes are categorized by number of passengers)
- Null values in injured and injured columns: are they possibly just 0 which haven't been explicitly entered into the data? Check reports and verify
- Effect of time of year on accidents?

---

## Data cleaning to-do list

1. Drop irrelevant columns
2. Deal with missing values
3. Check remaining columns if you want em
4. Clean up rows/intra-column clean-up: filter Investigation.Type to only accidents, Amateur.Built to only include non-amateur aircraft, (*maybe*) Number.of.Engine to only 1 & 2, 
5. Replace all null values in passenger numbers with 0 (assumption here is that all nulls are 0s that were not manually entered into data)
6. Clean up duplicates in the Make column

---

### 1. Column cleaning

In [82]:
# Dropping all irrelevant columns
irrelevant_columns = ['Event.Id', 'Accident.Number', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 
'Registration.Number', 'FAR.Description', 'Purpose.of.flight', 'Air.carrier', 'Report.Status', 'Publication.Date']

df = df.drop(columns=irrelevant_columns)
df.columns

Index(['Investigation.Type', 'Event.Date', 'Location', 'Country',
       'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category', 'Make',
       'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type',
       'Schedule', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight'],
      dtype='object')

In [83]:
# Checking for the number of missing values in our remaining columns
df.isna().sum()

Investigation.Type            0
Event.Date                    0
Location                     52
Country                     226
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
Schedule                  76307
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
dtype: int64

In [84]:
# Further pruning of columns with more than 25% of its data missing
df = df.drop(columns=['Aircraft.Category', 'Schedule', 'Broad.phase.of.flight'])
df.columns

Index(['Investigation.Type', 'Event.Date', 'Location', 'Country',
       'Injury.Severity', 'Aircraft.damage', 'Make', 'Model', 'Amateur.Built',
       'Number.of.Engines', 'Engine.Type', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition'],
      dtype='object')

In [85]:
# Renaming columns
new_column_names = {'Investigation.Type': 'Type', 'Event.Date': 'Date', 'Injury.Severity': 'Injury_Severity', 'Aircraft.damage': 'Damage_Type', 
'Number.of.Engines': 'Engines', 'Engine.Type': 'Engine_Type', 'Total.Fatal.Injuries': 'Fatal_Injuries', 'Total.Serious.Injuries': 'Serious_Injuries', 
'Total.Minor.Injuries': 'Minor_Injuries', 'Total.Uninjured': 'Uninjured', 'Weather.Condition': 'Weather', 'Amateur.Built': 'Ameteur_Built'}
df.rename(columns=new_column_names, inplace=True)

In [86]:
df.head()

,Type,Date,Location,Country,Injury_Severity,Damage_Type,Make,Model,Ameteur_Built,Engines,Engine_Type,Fatal_Injuries,Serious_Injuries,Minor_Injuries,Uninjured,Weather
0,Accident,1948-10-24,"MOOSE CREEK, ID",United States,Fatal(2),Destroyed,Stinson,108-3,No,1.0,Reciprocating,2.0,0.0,0.0,0.0,UNK
1,Accident,1962-07-19,"BRIDGEPORT, CA",United States,Fatal(4),Destroyed,Piper,PA24-180,No,1.0,Reciprocating,4.0,0.0,0.0,0.0,UNK
2,Accident,1974-08-30,"Saltville, VA",United States,Fatal(3),Destroyed,Cessna,172M,No,1.0,Reciprocating,3.0,NaN,NaN,NaN,IMC
3,Accident,1977-06-19,"EUREKA, CA",United States,Fatal(2),Destroyed,Rockwell,112,No,1.0,Reciprocating,2.0,0.0,0.0,0.0,IMC
4,Accident,1979-08-02,"Canton, OH",United States,Fatal(1),Destroyed,Cessna,501,No,NaN,NaN,1.0,2.0,NaN,0.0,VMC


#### 1.1 Injury_Severity pruning?

In [87]:
df['Fatal_Injuries'].sum()

50201.0

In [88]:
df['Fatality'] = df['Injury_Severity'].str.extract(r'\((\d+)\)')

df['Fatality'] = pd.to_numeric(df['Fatality'])

df['Fatality'].sum()

35617.0

In [89]:
# Checking if there are rows where Fatal_Injuries contains a 0 and Fatality is bigger than 0
df.loc[(df['Fatal_Injuries'] == 0) & (df['Fatality'] > 0)]

,Type,Date,Location,Country,Injury_Severity,Damage_Type,Make,Model,Ameteur_Built,Engines,Engine_Type,Fatal_Injuries,Serious_Injuries,Minor_Injuries,Uninjured,Weather,Fatality


In [90]:
# Checking if there are rows where Fatal_Injuries is bigger than 0 while Fatality is 0
df.loc[(df['Fatal_Injuries'] > 0) & (df['Fatality'] == 0)]

,Type,Date,Location,Country,Injury_Severity,Damage_Type,Make,Model,Ameteur_Built,Engines,Engine_Type,Fatal_Injuries,Serious_Injuries,Minor_Injuries,Uninjured,Weather,Fatality


In [91]:
# Checking if there are rows where Fatal_Injuries is null than 0 while Fatality is not null
df.loc[(df['Fatal_Injuries'].isna()) & (df['Fatality'].notna())]

,Type,Date,Location,Country,Injury_Severity,Damage_Type,Make,Model,Ameteur_Built,Engines,Engine_Type,Fatal_Injuries,Serious_Injuries,Minor_Injuries,Uninjured,Weather,Fatality


In [92]:
# Checking if there are rows where Fatal_Injuries is not null than 0 while Fatality is null
df.loc[(df['Fatal_Injuries'].notna()) & (df['Fatality'].isna())]

,Type,Date,Location,Country,Injury_Severity,Damage_Type,Make,Model,Ameteur_Built,Engines,Engine_Type,Fatal_Injuries,Serious_Injuries,Minor_Injuries,Uninjured,Weather,Fatality
7,Accident,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Cessna,140,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,NaN
8,Accident,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Cessna,401B,No,2.0,Reciprocating,0.0,0.0,0.0,2.0,IMC,NaN
9,Accident,1982-01-01,"JACKSONVILLE, FL",United States,Non-Fatal,Substantial,North American,NAVION L-17B,No,1.0,Reciprocating,0.0,0.0,3.0,0.0,IMC,NaN
10,Accident,1982-01-01,"HOBBS, NM",United States,Non-Fatal,Substantial,Piper,PA-28-161,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,NaN
11,Accident,1982-01-01,"TUSKEGEE, AL",United States,Non-Fatal,Substantial,Beech,V35B,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,Accident,2022-12-26,"Annapolis, MD",United States,Minor,NaN,PIPER,PA-28-151,No,NaN,NaN,0.0,1.0,0.0,0.0,NaN,NaN
88885,Accident,2022-12-26,"Hampton, NH",United States,NaN,NaN,BELLANCA,7ECA,No,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
88886,Accident,2022-12-26,"Payson, AZ",United States,Non-Fatal,Substantial,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1.0,NaN,0.0,0.0,0.0,1.0,VMC,NaN
88887,Accident,2022-12-26,"Morgan, UT",United States,NaN,NaN,CESSNA,210N,No,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [93]:
df.loc[(df['Fatal_Injuries'] == 0) & (df['Fatality'].isna())]

,Type,Date,Location,Country,Injury_Severity,Damage_Type,Make,Model,Ameteur_Built,Engines,Engine_Type,Fatal_Injuries,Serious_Injuries,Minor_Injuries,Uninjured,Weather,Fatality
7,Accident,1982-01-01,"PULLMAN, WA",United States,Non-Fatal,Substantial,Cessna,140,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,NaN
8,Accident,1982-01-01,"EAST HANOVER, NJ",United States,Non-Fatal,Substantial,Cessna,401B,No,2.0,Reciprocating,0.0,0.0,0.0,2.0,IMC,NaN
9,Accident,1982-01-01,"JACKSONVILLE, FL",United States,Non-Fatal,Substantial,North American,NAVION L-17B,No,1.0,Reciprocating,0.0,0.0,3.0,0.0,IMC,NaN
10,Accident,1982-01-01,"HOBBS, NM",United States,Non-Fatal,Substantial,Piper,PA-28-161,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,NaN
11,Accident,1982-01-01,"TUSKEGEE, AL",United States,Non-Fatal,Substantial,Beech,V35B,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,Accident,2022-12-26,"Annapolis, MD",United States,Minor,NaN,PIPER,PA-28-151,No,NaN,NaN,0.0,1.0,0.0,0.0,NaN,NaN
88885,Accident,2022-12-26,"Hampton, NH",United States,NaN,NaN,BELLANCA,7ECA,No,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
88886,Accident,2022-12-26,"Payson, AZ",United States,Non-Fatal,Substantial,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1.0,NaN,0.0,0.0,0.0,1.0,VMC,NaN
88887,Accident,2022-12-26,"Morgan, UT",United States,NaN,NaN,CESSNA,210N,No,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [94]:
# 
df_temp = df.loc[(df['Fatal_Injuries']) != (df['Fatality'])]

In [95]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76325 entries, 5 to 88888
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Type              76325 non-null  object 
 1   Date              76325 non-null  object 
 2   Location          76293 non-null  object 
 3   Country           76159 non-null  object 
 4   Injury_Severity   75325 non-null  object 
 5   Damage_Type       73240 non-null  object 
 6   Make              76270 non-null  object 
 7   Model             76246 non-null  object 
 8   Ameteur_Built     76256 non-null  object 
 9   Engines           71113 non-null  float64
 10  Engine_Type       69796 non-null  object 
 11  Fatal_Injuries    64924 non-null  float64
 12  Serious_Injuries  66468 non-null  float64
 13  Minor_Injuries    67349 non-null  float64
 14  Uninjured         73405 non-null  float64
 15  Weather           72024 non-null  object 
 16  Fatality          0 non-null      float64
dty

In [96]:
df_temp['Fatality'].sum()

0.0

In [97]:
df_temp['Fatal_Injuries'].sum()

14584.0

There is no extra information in the Injury_Severity column that is missing from Fatal_Injuries. In fact, Fatal_Injuries contains more information than the numbers stripped from the Injury_Severity column. The temp column Fatality can be thus dropped and the numbers in brackets can be stripped from the Injurity_Severity column so that only "Fatal" remains.

In [98]:
df = df.drop(columns=['Fatality'])

In [103]:
# Stripping the bracket enclosed numbers from the Injury_Severity column
df['Injury_Severity'] = df['Injury_Severity'].str.replace(r'Fatal\(\d+\)', 'Fatal', regex=True)

### 2. Dealing with missing values

In [105]:
df.isna().sum()

Type                    0
Date                    0
Location               52
Country               226
Injury_Severity      1000
Damage_Type          3194
Make                   63
Model                  92
Ameteur_Built         102
Engines              6084
Engine_Type          7096
Fatal_Injuries      11401
Serious_Injuries    12510
Minor_Injuries      11933
Uninjured            5912
Weather              4492
dtype: int64

#### 2.1 Replacing all missing values from Fatal_Injuries, Serious_Injuries, Minor_Injuries, Uninjured with 0 and creating a new column Total_Passengers

In [110]:
# Replacing missing values with 0
df['Fatal_Injuries'] = df['Fatal_Injuries'].replace({np.nan: 0})
df['Serious_Injuries'] = df['Serious_Injuries'].replace({np.nan: 0})
df['Minor_Injuries'] = df['Minor_Injuries'].replace({np.nan: 0})
df['Uninjured'] = df['Uninjured'].replace({np.nan: 0})

In [111]:
df.isna().sum()

Type                   0
Date                   0
Location              52
Country              226
Injury_Severity     1000
Damage_Type         3194
Make                  63
Model                 92
Ameteur_Built        102
Engines             6084
Engine_Type         7096
Fatal_Injuries         0
Serious_Injuries       0
Minor_Injuries         0
Uninjured              0
Weather             4492
dtype: int64

In [113]:
# Creating a Total_Passengers column
df['Total_Passengers'] = df['Fatal_Injuries'] + df['Serious_Injuries'] + df['Minor_Injuries'] + df['Uninjured']

#### 2.2 Dropping rows